### Step 2: Import Libraries

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)

import torch

### Step 3: Load Clean Dataset

In [9]:
df = pd.read_csv("../dataset/cleaned/clean_news.csv")
df.columns

Index(['title', 'text', 'label', 'clean_text'], dtype='object')

### Step 4: Train-Test Split

In [10]:
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

# Fix column name if needed
if "text" not in df.columns and "clean_text" in df.columns:
    df["text"] = df["clean_text"]

# Ensure text values are string
train_df["text"] = train_df["text"].astype(str).fillna("")
test_df["text"] = test_df["text"].astype(str).fillna("")

### Step 5: Convert to HuggingFace Dataset

In [11]:

from datasets import Dataset

train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

### Step 6: Tokenization (DistilBERT)

In [12]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )

train_ds = train_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

Map:   0%|          | 0/35935 [00:00<?, ? examples/s]

Map:   0%|          | 0/8984 [00:00<?, ? examples/s]

In [13]:
train_ds = train_ds.rename_column("label", "labels")
test_ds = test_ds.rename_column("label", "labels")

train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

### Step 7: Load DistilBERT Model

In [14]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Step 8: Training Arguments

In [17]:
training_args = TrainingArguments(
    output_dir="../models/distilbert_fake_news",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=50,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds
)

print("Trainer initialized. Ready to train!")

Trainer initialized. Ready to train!


### Step 9: Train the Model

In [18]:
trainer.train()

print("Model trained!")

/opt/anaconda3/envs/newsenv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,0.000000,0.012576
2,0.000000,0.001432


/opt/anaconda3/envs/newsenv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Model trained!
